In [1]:
import semiolog as slg

In [2]:
semiotic = slg.Cenematic("abacus")

SLG: Checking config correctness...
SLG: Config correct!
SLG [I]: Dataset loaded from disk (dataset file)
SLG [I]: Vocabulary loaded from disk


In [10]:
semiotic.syntagmatic.build()

SLG: Tokenizing dataset...


100%|██████████| 3/3 [00:00<00:00, 34.80ba/s]


SLG: Tokenized corpus saved to models/abacus/syntagmas/tokenized
SLG: Corpus tokenized!


In [3]:
meno_corpus = semiotic.corpus.train.filter(lambda paragraph: " m." in paragraph["text"])

In [4]:
sent_sq = semiotic(meno_corpus[0]["text"])

In [9]:
sent_sq.chain.input

"c.82 v [?] Sono 3 che ànno denari e trovano che ciascuno moltipli_ ca li suoi in sé, sonano insieme tutti e quadrati: sono 84, di poi somano insieme li denari che ànno ciascuno, tro vano, somano 14, si dimanda quanti denari avevano per uno; questa non vole dire altro sì no fami di 14 3 parti che so mati insieme li quadrati di ciascuna parte facino 84; farai cossi: pone la prima sia 1 co, la [seconda'] sia 4 in nunm ro, la terza sarà 10 m. 1 co, ora moltiplica la 2"

In [7]:
sent_sq.chain.labels

['c',
 '.',
 '82',
 'v',
 '[?]',
 'Sono',
 '3',
 'che',
 'ànno',
 'denari',
 'e',
 'trovano',
 'che',
 'ciascuno',
 'moltipli_',
 'ca',
 'li',
 'suoi',
 'in',
 'sé',
 ',',
 'sonano',
 'insieme',
 'tutti',
 'e',
 'quadrati',
 ':',
 'sono',
 '84',
 ',',
 'di',
 'poi',
 'somano',
 'insieme',
 'li',
 'denari',
 'che',
 'ànno',
 'ciascuno',
 ',',
 'tro',
 'vano',
 ',',
 'somano',
 '14',
 ',',
 'si',
 'dimanda',
 'quanti',
 'denari',
 'avevano',
 'per',
 'uno',
 ';',
 'questa',
 'non',
 'vole',
 'dire',
 'altro',
 'sì',
 'no',
 'fami',
 'di',
 '14',
 '3',
 'parti',
 'che',
 'so',
 'mati',
 'insieme',
 'li',
 'quadrati',
 'di',
 'ciascuna',
 'parte',
 'facino',
 '84',
 ';',
 'farai',
 'cossi',
 ':',
 'pone',
 'la',
 'prima',
 'sia',
 '1',
 'co',
 ',',
 'la',
 '[',
 'seconda',
 '[UNK]',
 'sia',
 '4',
 'in',
 'nunm',
 'ro',
 ',',
 'la',
 'terza',
 'sarà',
 '10',
 'm',
 '.',
 '1',
 'co',
 ',',
 'ora',
 'moltiplica',
 'la',
 '2']

In [8]:
sent_sq.parad_chain.df()

,c_0,._1,82_2,v_3,[?]_4,Sono_5,3_6,che_7,ànno_8,denari_9,...,10_101,m_102,._103,1_104,co_105,",_106",ora_107,moltiplica_108,la_109,2_110
0,345651121,"redure,",xpiani,4950.,Diremo,"araj,","redure,",xpiani,Fourth,xpiani,...,65/243,xpiani,Fourth,4950.,"ca,",ioo,"essentials,",xpiani,Tholomeo,xpiani
1,radic,habbi,tray,*/3,2028,15/71,"ninth,",Chosta,xpiani,impossibile,...,primogenjto,65/243,s’acordano,"araj,",mettila,"sessaia,","murati,","araj,",4950.,di//raj
2,•vii»»,cxxviii,"araj,",xpiani,soptosopra.,xpiani,Fourth,ca.,"3.R.E,.p",<94v>,...,editing,follicellus,costorono,"rasgione,",patrone,Fourth,xpiani,enteramente,persone.,4950.
3,"egle,","araj,",soptosopra.,14ss:,xpiani,Fourth,enteramente,’sia,Missure.,4950.,...,góooooo,inventum,^v-,dedicate,"rasgione,",volglola,figura,"ca,",xpiani,Tholomeo
4,gnerlj,persone.,4/33.,65/243,Fourth,513/576,par-tirai,ioo,Tholomeo,497,...,vegnono,9/64,àj,"risposta,","cierchi,",«dovemo,4950.,editing,par-tirai,editing
5,góooooo,radic,3364,"risposta,","araj,",mortality,l'ocha.Di,debiti.,soptosopra.,"chariche,",...,144400,26/13,72/].,331652.,Missure.,"lucchesi,","3.R.E,.p",dell'uncie,nolo,góooooo
6,e_ì..,n’àj,chose:,uzzi.,editing,appellantur.,currentibus.,ragiongnere,enteramente,purses,...,"ca,",roof-ridge.,"18/24,",fuore.,11/24.,poxxa,2dr.,more,"sua,",6274/512
7,similia.,puro,punti.,"araj,",“thick,VII0,figura,dinominato,"araj,",Fourth,...,Abbiamo:,Fourth,essare,sailing,soptosopra.,e_ì..,"l’agugniamo,",V.l8.1.,"araj,",Wel/6
8,4950.,Fourth,ioo,Tholomeo,169708791451478t,manefest’è,cunctos,"araj,",exordium.,938284139/3127605625,...,"rasgione,","rasgione,",xpiani,novini,xpiani,"araj,","ca,",impossibile,Muovexj,obvious.
9,enteramente,"matricina,",ciensy.,"‘''""‘per","ciccolo,",Tholomeo,"multipricazione,","found,",mercatantj,"18/24,",...,more,"1/04,","seguita,",345651121,volglola,viene'de,28800.,volglola,Observations,aponte


In [17]:
bla = semiotic.corpus.train.map(lambda batch: " ".join(batch["text"]).split(), batched=True)

  0%|          | 0/22 [00:00<?, ?ba/s]


TypeError: Provided `function` which is applied to all elements of table returns a variable of type <class 'list'>. Make sure provided `function` returns a variable of type `dict` (or a pyarrow table) to update the dataset or `None` if you are only interested in side effects.

In [19]:
voc_freq.most_common(10)

[('e', 44384),
 ('che', 32952),
 ('di', 23077),
 ('per', 19866),
 ('la', 18826),
 ('a', 17363),
 ('in', 14322),
 ('è', 11211),
 ('de', 11204),
 ('et', 10569)]

In [9]:
voc_freq.most_common(30)

[('e', 44384),
 ('che', 32952),
 ('di', 23077),
 ('per', 19866),
 ('la', 18826),
 ('a', 17363),
 ('in', 14322),
 ('è', 11211),
 ('de', 11204),
 ('et', 10569),
 ('il', 9082),
 ('2', 8516),
 ('fa', 8021),
 ('3', 7916),
 ('via', 7915),
 ('lo', 7478),
 ('1', 7431),
 ('4', 7144),
 ('si', 6876),
 ('R', 6052),
 ('5', 5988),
 ('le', 5626),
 ('s.', 5580),
 ('del', 5577),
 ('thè', 5445),
 ('meno', 5178),
 ('d.', 5159),
 ('cioè', 4931),
 ('se', 4921),
 ('ne', 4885)]

In [17]:
sent = "I have made my plans and I must stick to them"
sent_seq = semiotic(sent)
sent_seq.chain.labels

['i',
 'have',
 'm',
 'ad',
 'em',
 'y',
 'pl',
 'an',
 'sand',
 'im',
 'ust',
 'st',
 'ic',
 'k',
 'tothe',
 'm']

In [19]:
sent_seq = semiotic(sent)
sent_seq.chain.labels

['ihave', 'made', 'my', 'plan', 'sand', 'i', 'must', 'st', 'ick', 'to', 'them']